Laboratorio: Vector Database

In [1]:
%%cmd
pip install -r requirements.txt

Microsoft Windows [Versi�n 10.0.26200.7462]
(c) Microsoft Corporation. Todos los derechos reservados.

(upcvectordatabase) c:\Users\Andres Huamanlazo\Desktop\BIG DATA VECTORES\httpupc-pre-202502-dn24-trabajofinal-grupo1\vector_workspaces_2 - copia\notebook>pip install -r requirements.txt
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-6.2.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached gitpython-3.1.45-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jsonschema-4.25.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached narwhals-2.13.0-py3-none-any.w

  You can safely remove it manually.


   ------------------------------------- -- 18/19 [streamlit]]-------- 13/19 [gitdb] ---------- 14/19 [jsonschema-specifications]---------- 14/19 [jsonschema-specifications]------------------------------- -------- 15/19 [gitpython]-------- 15/19 [gitpython]------------------------------- -------- 15/19 [gitpython][gitpython] --------------------------------- ------ 16/19 [jsonschema]--------------------------------- ------ 16/19 [jsonschema]--------------------------------- ------ 16/19 [jsonschema]------ 16/19 [jsonschema]16/19 [jsonschema] ---- 17/19 [altair]---- 17/19 [altair]---- 17/19 [altair]---- 17/19 [altair]---- 17/19 [altair]---- 17/19 [altair]------------------------------------- -- 18/19 [streamlit]------------------------------------- -- 18/19 [streamlit]------------------------------------- -- 18/19 [streamlit] ------------------------------------- -- 18/19 [streamlit]18/19 [streamlit]------------------------------------- -- 18/19 [streamlit]------------------------------

In [2]:
import pandas as pd
import csv
pd.options.mode.chained_assignment = None  # default='warn'

fashion_df = pd.read_csv(filepath_or_buffer="../dataset/rakuten_uk_fashion.tsv", delimiter='\t', quoting=csv.QUOTE_NONE, encoding='utf-8')
fashion_df = fashion_df.rename(columns={'link': 'product_page', 'image_link': 'product_image'})

Transformación de Datos

In [3]:
def get_product_name(product_page: str):
    product_split = product_page.split(sep='/')
    product_format = product_split[4].replace('-', ' ')
    return product_format.title()

def get_price_currency(price: str):
    return price.split(sep=' ')[1]

def get_price_amount(price: str):
    return price.split(sep=' ')[0]

def get_product_lookbook(additional_image: str):
    product_lookbook = None
    product_image_list = additional_image.split(sep=',')
    if len(product_image_list) > 0:
        product_lookbook = product_image_list[0]
    return product_lookbook

def get_category(product_type: str):
    category = 'No specific'
    try:
        product_type_list = product_type.split(sep=' > ')
        if (product_type_list[0].strip() == 'Women'):
            category = product_type_list[1].strip()          
        else:
            category = product_type_list[0].strip()
    except Exception as e:
        print(product_type)
    return category

def get_subcategory(product_type: str):
    subcategory = 'No specific'
    try:
        product_type_list = product_type.split(sep=' > ')
        if (product_type_list[0].strip() == 'Women'):
            subcategory = product_type_list[2].strip()
        else:
            subcategory = product_type_list[1].strip()
    except Exception as e:
        print(product_type)
    return subcategory

In [4]:
fashion_df['product_name'] = fashion_df['product_page'].apply(get_product_name)
fashion_df['product_lookbook'] = fashion_df['product_page'].apply(get_product_lookbook)
fashion_df['price_currency'] = fashion_df['price'].apply(get_price_currency)
fashion_df['price'] = fashion_df['price'].apply(get_price_amount)
fashion_df['category'] = fashion_df['product_type'].apply(get_category)
fashion_df['subcategory'] = fashion_df['product_type'].apply(get_subcategory)
fashion_df['color'] = fashion_df['color'].fillna('No specific')
fashion_df['material'] = fashion_df['material'].fillna('No specific')

In [5]:
fashion_selected_df = fashion_df[['id', 'product_name', 'brand', 'description', 'product_type', 'gender', 'category', 'subcategory', 'color', 'material', 'price', 'price_currency','product_page', 'product_image', 'product_lookbook']]

#Segmento de Clothing
fashion_clothing_segment = fashion_selected_df[fashion_df['category'].isin(['Clothing'])]
fashion_clothing_segment.to_csv(path_or_buf='../dataset/clothing_segment.csv', index=False, encoding='utf-8', doublequote=True, sep='|')

Tratamiento de Imagenes

In [6]:
import pandas as pd

fashion_clothing_segment = pd.read_csv("../dataset/clothing_segment.csv", sep='|', encoding='utf-8')

fashion_clothing_segment = fashion_clothing_segment.sort_values(by=["brand"], ascending=True)
fashion_clothing_group = fashion_clothing_segment.groupby(by='brand')
fashion_clothing_segment.reset_index(drop=True, inplace=True)
fashion_clothing_segment['item'] = fashion_clothing_segment['id'].astype(str) + "|" + fashion_clothing_segment['product_name'].astype(str) + "|" + fashion_clothing_segment['category'].astype(str) + "|" + fashion_clothing_segment['subcategory'].astype(str) +"|"+ fashion_clothing_segment["product_image"].astype(str)

In [7]:
fashion_clothing_segment_example = fashion_clothing_segment[0:1000]
fashion_clothing_segment_example.to_csv(path_or_buf='../dataset/clothing_segment_example.csv', index=False, encoding='utf-8', doublequote=True, sep='|')

In [8]:
from PIL import Image
from io import BytesIO
import requests
import cv2
import numpy as np
from tqdm import tqdm
from base64 import b64encode
from concurrent.futures import ThreadPoolExecutor, as_completed

def embedding_image_product(idx, item):   
        
    try:
        item_column = item.split('|')
        product_id = item_column[0]
        product_name = item_column[1]
        category = item_column[2]
        subcategory = item_column[3]
        product_image = item_column[4]
        product_result = 'Ninguno'
        texts = [product_name, category, subcategory]

        response = requests.get(product_image,  timeout=200)

        if response.status_code == 200:
            image = Image.open(BytesIO(response.content))
            white_background = Image.new("RGB", image.size, "white")
            image = image.convert("RGBA")
            white_background.paste(image, (0, 0), image)    
            img_byte_arr = BytesIO()
            white_background.convert("RGB").save(img_byte_arr, format='JPEG')
            image_bytes = img_byte_arr.getvalue()

            image_from_bytes = Image.open(BytesIO(image_bytes))
            image_from_bytes.save(f'../inventory/original_image/{product_id}.jpg')

            image_ori = open(f'../inventory/original_image/{product_id}.jpg', 'rb')
            image_ori_bytes = image_ori.read()
            
            #blob_client = container_client.get_blob_client(f'image_products/{product_id}.jpg')
            #image_response = img_byte_arr.getvalue()#blob_client.download_blob().content_as_bytes()
            nparr = np.frombuffer(image_ori_bytes, dtype=np.uint8)
            image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
            new_height = 800
            print_rect = False
            font_scale = 0.7
            font_thickness = 2
            text_position = 0
            height, width = image.shape[:2]
            new_width = int((new_height / height) * width)
            resized_image = cv2.resize(image, (new_width, new_height))
            gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
            contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contours = sorted(contours, key=cv2.contourArea, reverse=True)
            x, y, w, h = cv2.boundingRect(contours[0])
            if print_rect:
                cv2.rectangle(resized_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            font = cv2.FONT_HERSHEY_SIMPLEX
            text_heights = []
            for text in texts:
                (_, text_height), _ = cv2.getTextSize(text, font, font_scale, font_thickness)
                text_heights.append(text_height)

            text_x = x + int(new_width * text_position) + 10
            text_y = y + int(h) - sum(text_heights)//2
            outline_thickness = 2
            for text in texts:
                cv2.putText(resized_image, text, (text_x, text_y), font, font_scale, (0, 255, 0), outline_thickness, cv2.LINE_AA)
                text_y += text_height + 5

            file_extension = ".jpg"
            image_string = cv2.imencode(file_extension, resized_image)[1].tobytes()#.tostring()
            #Put object
            with open(f'../inventory/processed_image/{product_id}_new.jpg', 'wb') as f:
                f.write(image_string)
            product_result = 'Success'
        else:
            product_result = f'Error. {response.status_code}'
    except Exception as e:
        print(e)
        product_result = f'Error: {e}'
            
    return idx, product_result


def fetch_image_product(idx, item):
    return embedding_image_product(idx, item)


def run_product_image(product_df, max_workers=10):
    item_products = product_df['item'].tolist()
    result_list = [None] * len(item_products)  # Pre-allocate list for correct indexing
    
    with tqdm(total=len(item_products)) as pbar:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_index = {executor.submit(fetch_image_product, i, item): i for i, item in enumerate(item_products)}
            for future in as_completed(future_to_index):
                pbar.update(1)
                index, result = future.result()  # Get correct index and image
                result_list[index] = result
    product_df['result'] = result_list
    return product_df

In [9]:
result_df = run_product_image(fashion_clothing_segment_example, max_workers=6)

100%|██████████| 1000/1000 [01:35<00:00, 10.52it/s]
